"""Ecommerce shipping prediction is the process of estimating the wheteher the product reached on time. which is  based on various factors such as the origin and destination of the package, the shipping method selected by the customer, the carrier used for shipping, and any potential delays or issues that may arise during the shipping process.
Machine learning models can be used to make accurate predictions about shipping times based on historical data and real-time updates from carriers. These models may take into account factors such as weather conditions, traffic, and other external factors that can impact delivery times.
Over All Ecommerce shipping prediction is an important tool for ecommerce businesses that want to provide accurate delivery estimates to their customers and improve their overall customer experience."""

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier as skrfc
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, LogisticRegressionCV, SGDClassifier, RidgeClassifier
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error, mean_absolute_error, f1_score
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn import svm, linear_model
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier

In [2]:
data = pd.read_csv('Train.csv')
data.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   10999 non-null  int64 
 1   Warehouse_block      10999 non-null  object
 2   Mode_of_Shipment     10999 non-null  object
 3   Customer_care_calls  10999 non-null  int64 
 4   Customer_rating      10999 non-null  int64 
 5   Cost_of_the_Product  10999 non-null  int64 
 6   Prior_purchases      10999 non-null  int64 
 7   Product_importance   10999 non-null  object
 8   Gender               10999 non-null  object
 9   Discount_offered     10999 non-null  int64 
 10  Weight_in_gms        10999 non-null  int64 
 11  Reached.on.Time_Y.N  10999 non-null  int64 
dtypes: int64(8), object(4)
memory usage: 1.0+ MB


In [11]:
['Warehouse_block','Mode_of_Shipment','Product_importance','Gender','Reached.on.Time_Y.N','ID']

['Warehouse_block',
 'Mode_of_Shipment',
 'Product_importance',
 'Gender',
 'Reached.on.Time_Y.N',
 'ID']

In [3]:
# Label Mapping
label_map={}
for i in data.columns:
    if str(data[i].dtype) is 'object':
        temp={}
        cats=data[i].unique()
        for index in range(len(cats)):
            temp[cats[index]]=index
        label_map[i]=temp
        #labeling
        data[i]=data[i].map(temp)
label_map

<>:4: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:4: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\ashwi\AppData\Local\Temp\ipykernel_5716\1465132513.py:4: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if str(data[i].dtype) is 'object':


{'Warehouse_block': {'D': 0, 'F': 1, 'A': 2, 'B': 3, 'C': 4},
 'Mode_of_Shipment': {'Flight': 0, 'Ship': 1, 'Road': 2},
 'Product_importance': {'low': 0, 'medium': 1, 'high': 2},
 'Gender': {'F': 0, 'M': 1}}

In [13]:
data.corr()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
ID,1.000000,0.000078,0.002758,0.188998,-0.005722,0.196791,0.145369,-0.056137,-0.001695,-0.598278,0.278312,-0.411822
Warehouse_block,0.000078,1.000000,0.001093,-0.005694,-0.006191,0.012304,-0.003886,0.006375,0.006213,-0.001249,0.000904,0.000128
Mode_of_Shipment,0.002758,0.001093,1.000000,-0.010264,0.002590,0.006913,0.002700,-0.007679,-0.017499,-0.001303,0.003489,-0.007797
Customer_care_calls,0.188998,-0.005694,-0.010264,1.000000,0.012209,0.323182,0.180771,-0.058134,0.002545,-0.130750,-0.276615,-0.067126
Customer_rating,-0.005722,-0.006191,0.002590,0.012209,1.000000,0.009270,0.013179,0.003999,0.002775,-0.003124,-0.001897,0.013119
Cost_of_the_Product,0.196791,0.012304,0.006913,0.323182,0.009270,1.000000,0.123676,-0.046789,0.019759,-0.138312,-0.132604,-0.073587
Prior_purchases,0.145369,-0.003886,0.002700,0.180771,0.013179,0.123676,1.000000,0.027317,-0.009395,-0.082769,-0.168213,-0.055515
Product_importance,-0.056137,0.006375,-0.007679,-0.058134,0.003999,-0.046789,0.027317,1.000000,-0.002974,0.026023,0.093211,0.020517
Gender,-0.001695,0.006213,-0.017499,0.002545,0.002775,0.019759,-0.009395,-0.002974,1.000000,-0.011777,0.003573,0.004689
Discount_offered,-0.598278,-0.001249,-0.001303,-0.130750,-0.003124,-0.138312,-0.082769,0.026023,-0.011777,1.000000,-0.376067,0.397108


In [14]:
def check_outliers(arr):
    Q1 = np.percentile(arr, 25,interpolation = 'midpoint')
    Q3 = np.percentile(arr, 75,interpolation = 'midpoint')
    IQR = Q3 - Q1
    
    #Above Upper bound
    upper=Q3+1.5*IQR
    upper_array=np.array(arr>=upper)
    print(' '*3,len(upper_array[upper_array == True]),'are over the upper bound:',upper)

    #Below Lower bound
    lower=Q1-1.5*IQR
    lower_array=np.array(arr<=lower)
    print(' '*3,len(lower_array[lower_array == True]),'are less than the lower bound:',lower,'\n')

for i in data.drop(columns=[
                            'Warehouse_block','Mode_of_Shipment','Product_importance','Gender','Reached.on.Time_Y.N','ID'
                           ]).columns:
    if str(data[i].dtype)=='object':
        continue
    print(i)
    check_outliers(data[i])

Customer_care_calls
    0 are over the upper bound: 8.0
    0 are less than the lower bound: 0.0 

Customer_rating
    0 are over the upper bound: 7.0
    0 are less than the lower bound: -1.0 

Cost_of_the_Product
    0 are over the upper bound: 374.0
    0 are less than the lower bound: 46.0 

Prior_purchases
    1003 are over the upper bound: 5.5
    0 are less than the lower bound: 1.5 

Discount_offered
    2262 are over the upper bound: 19.0
    0 are less than the lower bound: -5.0 

Weight_in_gms
    0 are over the upper bound: 9865.75
    0 are less than the lower bound: -2976.25 



# Train test split

In [4]:
x_train, x_test, y_train, y_test = train_test_split(
                                        data.drop(columns=['ID','Reached.on.Time_Y.N']),
                                        data['Reached.on.Time_Y.N'],
                                        random_state=1234,test_size = 0.20,
                                        shuffle=True
                                        )
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(8799, 10)
(2200, 10)
(8799,)
(2200,)


# Training Function

In [69]:
def models_eval_sc(x_train,y_train,x_test,y_test):
    
    lg = make_pipeline(StandardScaler(),LogisticRegression(random_state=1234))
    lg.fit(x_train,y_train)
    print('--Logistic Regression')
    print('Train Score:',lg.score(x_train,y_train))
    print('Test Score:',lg.score(x_test,y_test))
    print()
    
    lcv = make_pipeline(StandardScaler(),LogisticRegressionCV(random_state=1234))
    lcv.fit(x_train,y_train)
    print('--Logistic Regression CV')
    print('Train Score:',lcv.score(x_train,y_train))
    print('Test Score:',lcv.score(x_test,y_test))
    print()
    
    print('--XGBoost')
    xgb = make_pipeline(StandardScaler(),XGBClassifier(n_estimators=300,n_jobs=-1,random_state=1234))
    xgb.fit(x_train,y_train)
    print('Train Score:',xgb.score(x_train,y_train))
    print('Test Score:',xgb.score(x_test,y_test))
    print()
    
    print('--Ridge Classifier')
    rg = make_pipeline(StandardScaler(),RidgeClassifier(random_state=1234))
    rg.fit(x_train,y_train)
    print('Train Score:',rg.score(x_train,y_train))
    print('Test Score:',rg.score(x_test,y_test))
    print()
    
    print('--KNN')
    knn = make_pipeline(StandardScaler(),KNeighborsClassifier())
    knn.fit(x_train,y_train)
    print('Train Score:',knn.score(x_train,y_train))
    print('Test Score:',knn.score(x_test,y_test))
    print()
    
    print('--Random Forest')
    rf = make_pipeline(StandardScaler(),RandomForestClassifier(random_state=1234))
    rf.fit(x_train,y_train)
    print('Train Score:',rf.score(x_train,y_train))
    print('Test Score:',rf.score(x_test,y_test))
    print()
    
    return lg,lcv,xgb,rg,knn,rf

In [70]:
def models_eval_mm(x_train,y_train,x_test,y_test):
    lg = LogisticRegression(random_state=1234)
    lg.fit(x_train,y_train)
    print('--Logistic Regression')
    print('Train Score:',lg.score(x_train,y_train))
    print('Test Score:',lg.score(x_test,y_test))
    print()
    
    lcv = LogisticRegressionCV(random_state=1234)
    lcv.fit(x_train,y_train)
    print('--Logistic Regression CV')
    print('Train Score:',lcv.score(x_train,y_train))
    print('Test Score:',lcv.score(x_test,y_test))
    print()
    
    print('--XGBoost')
    xgb = XGBClassifier(n_estimators=300,n_jobs=-1,random_state=1234) 
    xgb.fit(x_train,y_train)
    print('Train Score:',xgb.score(x_train,y_train))
    print('Test Score:',xgb.score(x_test,y_test))
    print()
    
    print('--Ridge Classifier')
    rg = RidgeClassifier(random_state=1234)
    rg.fit(x_train,y_train)
    print('Train Score:',rg.score(x_train,y_train))
    print('Test Score:',rg.score(x_test,y_test))
    print()
    
    print('--KNN')
    knn = KNeighborsClassifier()
    knn.fit(x_train,y_train)
    print('Train Score:',knn.score(x_train,y_train))
    print('Test Score:',knn.score(x_test,y_test))
    print()
    
    print('--Random Forest')
    rf = RandomForestClassifier(random_state=1234)
    rf.fit(x_train,y_train)
    print('Train Score:',rf.score(x_train,y_train))
    print('Test Score:',rf.score(x_test,y_test))
    print()
    return lg,lcv,xgb,rg,knn,rf

# Training after standard scaling

In [71]:
lg,lcv,xgb,rg,knn,rf = models_eval_sc(x_train,y_train,x_test,y_test)

--Logistic Regression
Train Score: 0.6403000340947835
Test Score: 0.6427272727272727

--Logistic Regression CV
Train Score: 0.6412092283214001
Test Score: 0.6381818181818182

--XGBoost
Train Score: 0.9895442663939084
Test Score: 0.6472727272727272

--Ridge Classifier
Train Score: 0.6504148198658939
Test Score: 0.6522727272727272

--KNN
Train Score: 0.7782702579838618
Test Score: 0.6286363636363637

--Random Forest
Train Score: 1.0
Test Score: 0.6559090909090909



In [72]:
sc = StandardScaler()
x_t = sc.fit_transform(x_train)
x_te = sc.fit_transform(x_test)
lg,lcv,xgb,rg,knn,rf = models_eval_sc(x_t,y_train,x_te,y_test)

--Logistic Regression
Train Score: 0.6403000340947835
Test Score: 0.6336363636363637

--Logistic Regression CV
Train Score: 0.6412092283214001
Test Score: 0.6363636363636364

--XGBoost
Train Score: 0.9895442663939084
Test Score: 0.644090909090909

--Ridge Classifier
Train Score: 0.6504148198658939
Test Score: 0.6554545454545454

--KNN
Train Score: 0.7782702579838618
Test Score: 0.6309090909090909

--Random Forest
Train Score: 1.0
Test Score: 0.6527272727272727



# l1 normalization

In [78]:
from sklearn.preprocessing import Normalizer
Data_normalizer = Normalizer(norm='l1').fit(x_train)
x_train_normalized = Data_normalizer.transform(x_train)
x_test_normalized = Data_normalizer.transform(x_test)
lg,lcv,xgb,rg,knn,rf =  models_eval_mm(x_train_normalized,y_train,x_test_normalized,y_test)

--Logistic Regression
Train Score: 0.5976815547221275
Test Score: 0.5927272727272728

--Logistic Regression CV
Train Score: 0.6362086600750085
Test Score: 0.6327272727272727

--XGBoost
Train Score: 0.9987498579384021
Test Score: 0.645

--Ridge Classifier
Train Score: 0.5976815547221275
Test Score: 0.5927272727272728

--KNN
Train Score: 0.7797476986021139
Test Score: 0.6327272727272727

--Random Forest
Train Score: 1.0
Test Score: 0.6718181818181819



In [79]:
y_pred = rf.predict(x_test_normalized)
from sklearn.metrics import classification_report, confusion_matrix

print('--->CLASSIFICATION REPORT=>\n\n',classification_report(y_test, y_pred))

print('--->CONFUSION MATRIX=>\n\n',confusion_matrix(y_test, y_pred))

--->CLASSIFICATION REPORT=>

               precision    recall  f1-score   support

           0       0.58      0.69      0.63       896
           1       0.76      0.66      0.70      1304

    accuracy                           0.67      2200
   macro avg       0.67      0.68      0.67      2200
weighted avg       0.69      0.67      0.67      2200

--->CONFUSION MATRIX=>

 [[621 275]
 [447 857]]


# Check Data Balance

In [85]:
len(data[data['Reached.on.Time_Y.N']==1]['Reached.on.Time_Y.N'])

6563

In [86]:
len(data[data['Reached.on.Time_Y.N']==0]['Reached.on.Time_Y.N'])

4436

# Hyper parameter optimisation for SVM
svc = svm.SVC(random_state=1234,kernel='rbf',C= 10, gamma= 5 , tol = 1e-2, verbose = 1) 0.6648482782134334 0.6704545454545454

In [13]:
from sklearn import svm
from sklearn.preprocessing import Normalizer
Data_normalizer = Normalizer(norm='l1').fit(x_train)
x_train_normalized = Data_normalizer.transform(x_train)
x_test_normalized = Data_normalizer.transform(x_test)

In [61]:
#svc = svm.SVC(random_state=1234,kernel='rbf',C= 6, gamma= 4) 0.6648482782134334 0.6672727272727272
#svc = svm.SVC(random_state=1234,kernel='rbf',C= 5, gamma= 5) 0.6640527332651438 0.6686363636363636
#svc = svm.SVC(random_state=1234,kernel='rbf',C= 10, gamma= 5, verbose = 3) 0.6648482782134334 0.6695454545454546
#svc = svm.SVC(random_state=1234,kernel='rbf',C= 10, gamma= 5 , tol = 1e-2, verbose = 1) 0.6648482782134334 0.6704545454545454
svc = svm.SVC(random_state=1234,kernel='rbf',C= 10, gamma= 5 , tol = 1e-2,verbose = 1)
svc.fit(x_train_normalized, y_train)
print('train score',svc.score(x_train_normalized,y_train))
print('test score',svc.score(x_test_normalized,y_test))

[LibSVM]train score 0.6648482782134334
test score 0.6704545454545454


In [56]:
svc = svm.SVC(random_state=1234,kernel='rbf')
params = {
            'C': [ 10, 13], 
              'gamma': [4,5],
    'tol':[1e-1,1e-2,1e-3]
        }
fitmodel = GridSearchCV(svc, param_grid=params, cv=5, refit=True, scoring="accuracy", n_jobs=-1, verbose=3)
fitmodel.fit(x_train_normalized, y_train)
print(fitmodel.best_estimator_, fitmodel.best_params_, fitmodel.best_score_)

# SVC(C=10, gamma=1, random_state=1234) {'C': 10, 'gamma': 1, 'kernel': 'rbf'} 0.6657575326890279
# SVC(C=6, gamma=2, random_state=1234) {'C': 6, 'gamma': 2, 'kernel': 'rbf'} 0.6659845470050132

Fitting 5 folds for each of 12 candidates, totalling 60 fits
SVC(C=10, gamma=4, random_state=1234) {'C': 10, 'gamma': 4, 'tol': 0.001} 0.665189027856737


In [30]:
svc = svm.SVC(random_state=1234)
params = {
        'kernel' : ['poly', 'rbf'],
            'C': [ 6,8,9,10], 
              'gamma': [2,1, 0.1],
        }
fitmodel = GridSearchCV(svc, param_grid=params, cv=5, refit=True, scoring="accuracy", n_jobs=-1, verbose=3)
fitmodel.fit(x_train_normalized, y_train)
print(fitmodel.best_estimator_, fitmodel.best_params_, fitmodel.best_score_)

# SVC(C=10, gamma=1, random_state=1234) {'C': 10, 'gamma': 1, 'kernel': 'rbf'} 0.6657575326890279
# SVC(C=6, gamma=2, random_state=1234) {'C': 6, 'gamma': 2, 'kernel': 'rbf'} 0.6659845470050132

Fitting 5 folds for each of 18 candidates, totalling 90 fits
SVC(C=6, gamma=2, random_state=1234) {'C': 6, 'gamma': 2, 'kernel': 'rbf'} 0.6659845470050132


# Hyper parameter optimisation for XGboost

In [43]:
xgb._get_params

<bound method _BaseComposition._get_params of Pipeline(steps=[('standardscaler', StandardScaler()),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.300000012, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=6, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=100, n_jobs=0, num_parallel_tree=1,
                               p

In [142]:
params = {
        'min_child_weight': [10,20],
        'gamma': [1.5, 2.0, 2.5],
        'colsample_bytree': [0.6, 0.8, 0.9],
        'max_depth': [4,5,6]
        }
xgb = XGBClassifier(learning_rate=0.5, n_estimators=100, objective='binary:logistic', nthread=3)
fitmodel = GridSearchCV(xgb, param_grid=params, cv=5, refit=True, scoring="accuracy", n_jobs=-1, verbose=3)
fitmodel.fit(x_train_normalized, y_train)
print(fitmodel.best_estimator_, fitmodel.best_params_, fitmodel.best_score_)

# {'colsample_bytree': 0.8, 'gamma': 2.0, 'max_depth': 4, 'min_child_weight': 10} 0.6608713628611298

Fitting 5 folds for each of 54 candidates, totalling 270 fits
XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=2.0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.5, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=10,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=3, nthread=3, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, ...) {'colsample_bytree': 0.8, 'gamma': 2.0, 'max_depth': 4, 'min_child_weight': 10} 0.6608713628611298


In [105]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 1234,
 'verbose': 0,
 'warm_start': False}

# Random Forest Hyperparameter optimisation

In [108]:
# Plug in appropriate max_depth and random_state parameters
rf = RandomForestClassifier()
rf_param_grid = {
    'n_estimators': [200,300,500], 
    'criterion': ['entropy','gini'],
    'max_depth': [7,8,60,80,100],
    'max_features': ['auto', 'sqrt', 'log2']
}
rf_cv= GridSearchCV(rf,rf_param_grid,cv=7,scoring="accuracy", n_jobs=-1, verbose=3)
rf_cv.fit(x_train,y_train)

print("Best Score:" + str(rf_cv.best_score_))
print("Best Parameters: " + str(rf_cv.best_params_))

#Best Score:0.6849642004773271
#Best Parameters: {'criterion': 'entropy', 'max_depth': 7, 'max_features': 'auto', 'n_estimators': 200}

Fitting 7 folds for each of 90 candidates, totalling 630 fits


C:\Users\ashwi\AppData\Roaming\Python\Python39\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best Score:0.6849642004773271
Best Parameters: {'criterion': 'entropy', 'max_depth': 7, 'max_features': 'auto', 'n_estimators': 200}


In [127]:
# Plug in appropriate max_depth and random_state parameters
rf = RandomForestClassifier(max_features= 'auto',criterion= 'entropy')
rf_param_grid = {
    'n_estimators': [100,150,200],
    'max_depth': [5,6,7],
    'min_samples_leaf':[1,3,5]
}
rf_cv= GridSearchCV(rf,rf_param_grid,cv=6,scoring="accuracy", n_jobs=-1, verbose=3)
rf_cv.fit(x_train_normalized,y_train)

print("Best Score:" + str(rf_cv.best_score_))
print("Best Parameters: " + str(rf_cv.best_params_))

Fitting 6 folds for each of 27 candidates, totalling 162 fits


C:\Users\ashwi\AppData\Roaming\Python\Python39\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best Score:0.6806471957105122
Best Parameters: {'max_depth': 7, 'min_samples_leaf': 5, 'n_estimators': 100}


In [145]:
# DEPLOYING model
rf = RandomForestClassifier(criterion= 'entropy', max_depth= 6, max_features= 'auto', n_estimators= 100)

rf.fit(x_train_normalized,y_train)
print('train',rf.score(x_train_normalized,y_train))
print('test',rf.score(x_test_normalized,y_test))

C:\Users\ashwi\AppData\Roaming\Python\Python39\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


train 0.7034890328446415
test 0.6872727272727273


# Logistic regression hyperparameter optimisation

In [141]:
# Plug in appropriate max_depth and random_state parameters
lg = LogisticRegressionCV(n_jobs=-1,random_state= 1234)
lg_param_grid = {
    'Cs': [6,8,10,15,20],
     'max_iter': [60,80,100]
}
lg_cv= GridSearchCV(lg,lg_param_grid,cv=5,scoring="accuracy", n_jobs=-1, verbose=3)
lg_cv.fit(x_train_normalized,y_train)

print("Best Score:" + str(lg_cv.best_score_))
print("Best Parameters: " + str(lg_cv.best_params_))

Fitting 5 folds for each of 15 candidates, totalling 75 fits
Best Score:0.633821644529433
Best Parameters: {'Cs': 10, 'max_iter': 60}


In [130]:
lcv.get_params()

{'Cs': 10,
 'class_weight': None,
 'cv': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1.0,
 'l1_ratios': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 1234,
 'refit': True,
 'scoring': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0}

# Saving model

In [143]:
import pickle as pkl

In [146]:
pkl.dump(rf, open('rf_acc_68.pkl', 'wb'))

# Load model

In [147]:
model = pkl.load(open('rf_acc_68.pkl', 'rb'))

In [148]:
x_test_normalized

array([[0.00000000e+00, 4.60829493e-04, 2.76497696e-03, ...,
        4.60829493e-04, 4.60829493e-03, 8.59907834e-01],
       [8.71080139e-04, 4.35540070e-04, 1.74216028e-03, ...,
        0.00000000e+00, 3.48432056e-03, 8.71080139e-01],
       [1.61943320e-04, 3.23886640e-04, 4.85829960e-04, ...,
        0.00000000e+00, 1.13360324e-03, 9.65668016e-01],
       ...,
       [7.09723208e-04, 0.00000000e+00, 4.25833925e-03, ...,
        0.00000000e+00, 4.96806246e-03, 7.65081618e-01],
       [8.93655049e-04, 2.23413762e-04, 1.34048257e-03, ...,
        0.00000000e+00, 1.78731010e-03, 9.47944593e-01],
       [2.32378002e-03, 1.54918668e-03, 3.87296669e-03, ...,
        0.00000000e+00, 2.32378002e-03, 7.78466305e-01]])

In [160]:
x_test_normalized[0].reshape(1,-1).shape

(1, 10)

In [159]:
model.predict(x_test_normalized[0].reshape(1,-1))

array([0], dtype=int64)

In [162]:
y_test

7212    1
7220    0
4637    1
2709    1
8161    0
       ..
1628    1
901     1
8903    0
7018    1
8349    1
Name: Reached.on.Time_Y.N, Length: 2200, dtype: int64

In [ ]:
'ID', 'Warehouse_block', 'Mode_of_Shipment', 'Customer_care_calls',
       'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases',
       'Product_importance', 'Gender', 'Discount_offered', 'Weight_in_gms',
       'Reached.on.Time_Y.N'

In [167]:
import pickle
from flask import Flask , request, render_template
app = Flask(__name__)
model = pickle.load(open("rf_acc_68.pkl","rb"))

@app.route('/')
def indput():
    return render_template('input.html')

@app.route('/predict',methods = ['GET','POST'])
def admin():
    Warehouse_block=eval(request.form["Warehouse_block"])
    Mode_of_Shipment=eval(request.form["Mode_of_Shipment"])
    Customer_care_calls=eval(request.form["Customer_care_calls"])
    Customer_rating=eval(request.form["Customer_rating"])
    Cost_of_the_Product = eval(request.form["Cost_of_the_Product"])
    Prior_purchases = eval(request.form["Prior_purchases"])
    Product_importance = eval(request.form["Product_importance"])
    Gender = eval(request.form["Gender"])
    Discount_offered = eval(request.form["Discount_offered"])
    Weight_in_gms = eval(request.form["Weight_in_gms"])
    
    preds=[[Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,
           Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms]]
    xx=model.predict(preds)
    print(xx)
    if xx[0]==1:
        return render_template("input.html",p="Reached on time")
    return render_template("input.html",p="Not reached on time")
if __name__ == '__main__':
    app.run(debug = False, port=4000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Feb/2023 16:07:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Feb/2023 16:08:18] "POST /predict HTTP/1.1" 200 -


[1]


127.0.0.1 - - [27/Feb/2023 16:09:31] "POST /predict HTTP/1.1" 200 -


[1]
